In [39]:
from os import path
import re
# importing the requests library 
import requests 
import json
import ast

In [23]:
import pandas as pd
corpus_df = pd.read_csv('/home/iftekhar/amiebot/Resources/876/data/corpus.csv')

In [24]:
corpus_df.columns

Index(['page', 'file', 'file_page', 'url', 'title', 'text'], dtype='object')

In [99]:
if 'page' not in corpus_df.columns:
    print('ok')

In [29]:
test_df = pd.read_csv('/home/iftekhar/Documents/test.csv')
urls = test_df.iloc[:, 1:].values.tolist()
questions = test_df['question'].values.tolist()

In [ ]:
urls

In [30]:
questions

['dカーシェアで利用できるカーシェアサービスは何がありますか？',
 'dカーシェアで利用できるカーシェアサービス',
 'カーシェアサービスは何がありますか？',
 'オリックスカーシェア',
 'ｄカーシェアで予約']

In [8]:
corpus_df_url = corpus_df.url.values.tolist()

In [28]:
from collections import defaultdict
test_corpus = defaultdict(list)
count = 0
for url_list in urls:
    for url in url_list:
        # print(url)
        try:
            page_id = list(set(corpus_df[(corpus_df['url'] == url)].page.values))[0]
            test_corpus['page'].append(page_id)
            test_corpus['url'].append(url)
            test_corpus['question'].append(questions[count])
        except IndexError:
            print(url, 'url not found')
    count += 1

NameError: name 'questions' is not defined

In [ ]:
pd.DataFrame(test_corpus)

In [122]:
a=1

In [124]:
b = [a]

### Starting of classification Tag evaluation API

In [31]:
test_df = pd.read_csv('/home/iftekhar/Downloads/test.csv')

In [32]:
test_df.head()

,url,question,tag
0,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,利用規約に違反した場合、どの様な対応が取られますか？,なし
1,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,料金の未払い,なし
2,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,サービスの利用停止,なし
3,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,利用規約に違反,なし
4,https://d.dmkt-sp.jp/common/qa/category/dmarke...,dマーケットとはどのようなサイトですか？,dマーケット


In [65]:
url_list = []
ques = []
ind = 1
collect = []
for index, col in test_df.iterrows():
    url_list.append(col['url'])
    ques.append(col['question'])
    page_id = list(set(corpus_df[(corpus_df['url'] == col['url'])].page.values))
    # print(col['url'], page_id, col['question'], col['tag'])
    if page_id:
        collect.append(page_id[0])
    else:
        collect.append(99999)
    ind += 1

In [ ]:
collect

In [61]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"

result = []
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/iftekhar/amiebot/Resources/876/model', 
            'data_dir':'/home/iftekhar/amiebot/Resources/876/data', 
            'agent_id':-1, 
            'query':col['question'], 
            'max_answer':3,
            'classification': col['tag']
           } 
    
    r = requests.post(url = API_ENDPOINT, data = data) 
    result.append(r.text)
    

In [62]:
len(result)

154

In [ ]:
result

In [203]:
r = json.dumps(result)

In [204]:
file = open(path.join('/home/iftekhar/Downloads/', "output.json"), "w", encoding='utf-8')
json.dump(r, file, ensure_ascii=False)
file.close()
    

In [67]:
saver = {}
i = 0
for items in result:
    p = ast.literal_eval(json.loads(items))
    saver[i] = {} 
    for items in p['page_relevance']:
        # print(items['doc_id'], items['keywords'])
        saver[i]['actual'] = collect[i]
        saver[i][items['doc_id']] = items['keywords']
    i += 1

In [68]:
saver

{0: {'actual': 99999,
  2123: ['利用規約', '違反', '様', '対応'],
  1263: ['利用規約', '違反', '様', '対応'],
  2329: ['利用規約', '違反', '様']},
 1: {'actual': 99999,
  2123: ['料金', '未払い'],
  1263: ['料金', '未払い'],
  1024: ['料金', '未払い']},
 2: {'actual': 99999,
  1182: ['サービス', '利用', '停止'],
  197: ['サービス', '利用', '停止'],
  2123: ['サービス', '利用', '停止']},
 3: {'actual': 99999,
  2123: ['利用規約', '違反'],
  1263: ['利用規約', '違反'],
  2329: ['利用規約', '違反']},
 4: {'actual': 134,
  134: ['dマーケット', 'dマーケット', 'サイト', 'dマーケット', 'サイト'],
  44: ['dマーケット', 'dマーケット', 'サイト', 'dマーケット', 'サイト'],
  46: ['dマーケット', 'dマーケット', 'サイト', 'dマーケット', 'サイト']},
 5: {'actual': 134,
  158: ['dマーケット', 'dマーケット', 'dマーケット'],
  190: ['dマーケット', 'dマーケット', 'dマーケット'],
  144: ['dマーケット', 'dマーケット', 'dマーケット']},
 6: {'actual': 134,
  134: ['dマーケット', 'dマーケット', 'サイト', 'dマーケット', 'サイト'],
  44: ['dマーケット', 'dマーケット', 'サイト', 'dマーケット', 'サイト'],
  46: ['dマーケット', 'dマーケット', 'サイト', 'dマーケット', 'サイト']},
 7: {'actual': 134,
  158: ['dマーケット', 'dマーケット', 'dマーケット'],
  190: ['dマーケット', 'dマーケット'

In [69]:
j = 0
s = 0
for items in saver.values():
    #print(items)
    if items:
        z = [x for x in items.keys()][1:]
        score = mean_reciprocal_rank_score(items['actual'], z)
        if not score > 0:
            s += 1
            print("Not Found: ", ques[j], "\nCorrect", url_list[j])
            for ii in z:
                print('Predict:', ' '.join(list(set(corpus_df[(corpus_df['page'] == ii)].url.values))))
        print(j,':', items['actual'], '->', z, '->', score)
    j += 1
print('Failed ', s )

Not Found:  利用規約に違反した場合、どの様な対応が取られますか？ 
Correct https://faq.carshare.dmkt-sp.jp/faq/show/9?back=front%2Fcategory%3Ashow&category_id=10&page=1&site_domain=default&sort=sort_access&sort_order=descc
Predict: https://faq.carshare.dmkt-sp.jp/print/faq/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/faq/show/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/print/faq/336?site_domain=default
0 : 99999 -> [2123, 1263, 2329] -> 0
Not Found:  料金の未払い 
Correct https://faq.carshare.dmkt-sp.jp/faq/show/9?back=front%2Fcategory%3Ashow&category_id=10&page=1&site_domain=default&sort=sort_access&sort_order=descc
Predict: https://faq.carshare.dmkt-sp.jp/print/faq/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/faq/show/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/category/show/2?site_domain=default
1 : 99999 -> [2123, 1263, 1024] -> 0
Not Found:  サービスの利用停止 
Correct https://faq.carshare.dmkt-sp.jp/faq/show/9?back=front%2Fcategory%3Ashow&category_

In [37]:
def mean_reciprocal_rank_score(actual_value, predicted_values):
    pos = 0
    val = 0
    for i in predicted_values:
        if i == actual_value and pos == 0:
            val = 1
            break
        elif i == actual_value and pos == 1:
            val = 0.5
            break
        elif i == actual_value and pos == 2:
            val = 0.33
            break
        else:
            val = 0
        pos += 1

    return val

In [347]:
h = []
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"

result = []
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/iftekhar/amiebot/Resources/876/model', 
            'data_dir':'/home/iftekhar/amiebot/Resources/876/data', 
            'agent_id':-1, 
            'query':col['question'], 
            'max_answer':3
           } 
    
    r = requests.post(url = API_ENDPOINT, data = data) 
    h.append(r.text)

In [56]:
a = 'A'
b = 'A'
if a in b:
    print('ok')

ok
